# ===========================================================
# 06 Working with Data

## Objectives
- Select individual values from a Pandas dataframe.
- Select entire rows or entire columns from a dataframe.
- Select a subset of both rows and columns from a dataframe in a single operation.
- Filter data based on values in a dataframe.
- Apply a split-apply-combine workflow to a dataframe.
- Convert a dataframe between wide and long data formats.

## Some Notes on DataFrames

- A DataFrame is a collection of Series
  - The DataFrame is the way Pandas represents a table, and Series is the data-structure
    Pandas use to represent a column.
- Pandas is built on top of the Numpy library
    - We get to use those function too
- Benefits of using Pandas:
  - Interface to access individual records
  - Proper handling of missing values
  - Relational-databases operations between DataFrames

## Selecting values
- Use `DataFrame.iloc[ROW, COLUMN]` to select values by their numerical position

In [ ]:
import pandas as pd
data = pd.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.iloc[0, 0])

- Use `DataFrame.loc[..., ...]` to select values by their row/column labels.

In [ ]:
data = pd.read_csv('data/gapminder_gdp_europe.csv', index_col='country')
print(data.loc["Albania", "gdpPercap_1952"])

- Use `:` on its own to mean all columns or all rows.
  - This follows Python's slicing notation.

In [ ]:
print(data.loc["Albania", :])

- We get the same result printing `data.loc["Albania"]` (without a second index).

In [ ]:
print(data.loc[:, "gdpPercap_1952"])

- Would get the same result printing `data["gdpPercap_1952"]`
- Slicing works with labels as well as numerical positions

In [ ]:
# 1. Use slicing to select GDP data from Italy to Poland
#    and from 1962 to 1972.j
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'])

- Slicing using `loc` is inclusive at both ends
  - This differs from slicing using `iloc`  

## Putting it Together
- The result from slicing can be used in further operations.

In [ ]:
# Find the maximum GDP for the above countries and above years
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].max())

In [ ]:
# Find the minimum GDP for the above countries and above years
print(data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972'].min())

## Filtering Data
- Use comparisons to select data based on values.
- Comparisons are applied element by element.
- They return a similarly-shaped dataframe of `True` and `False` values.

In [ ]:
# Use a subset of data to keep output readable.
subset = data.loc['Italy':'Poland', 'gdpPercap_1962':'gdpPercap_1972']
print('Subset of data:\n', subset)

# Which values were greater than 10000 ?
print('\nWhere are values large?\n', subset > 10000)

- A dataframe full of Booleans is sometimes called a *mask* because of how it can be used.

In [ ]:
mask = subset > 10000
print(subset[mask])

- NaNs (Not a Number) are ignored by operations like max, min, average, etc.

In [ ]:
print(subset[subset > 10000].describe())

## Exercise: Split-Apply-Combine

### Group By

- A common data-wrangling technique is the split-apply-combine technique
![Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/_images/06_groupby1.svg)

Source: <https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html#min-tut-06-stats>

## GDP of European Countries
We will practice this technique with the Gapminder data. Suppose we want to have a clearer view on how the European countries split themselves according to their GDP.

1.  We will split the countries in two groups during the years surveyed,
    those who presented a GDP *higher* than the European average and those with a *lower* GDP.
2.  We then estimate a *wealthy score* based on the historical (from 1962 to 2007) values,
    where we count how many times a country participated in the higher GDP group and give a percentage score.
3.  We will determine the totol GDP contribution for each year using wealth scores as categories.

**Part I - Split/Apply**

Type the following code and run it.

```python
wealth_score = data[data > data.mean()].count(axis=1) / len(data.columns)
wealth_score
```

- We filtered all the countris that were above the mean GDP.
- We then counted all the non-NaN values.
  - Our filter used NaN's for years a country was equal to or less than the mean.
- `axis` tells `count()` whether or not to count across rows or columns.
  - `0` means count non-NaN's in each column
  - `1` means count non-NaN's in each row (what we did)
  - I like to think about what is being collapsed (see picture below)
  - ... or just memorize 0 = down, 1 = across
- Finally, we divided the counts by the number of years (columns) to get a percentage score,

![axis](https://i.stack.imgur.com/DL0iQ.jpg)

Source: <https://stackoverflow.com/questions/25773245/ambiguity-in-pandas-dataframe-numpy-array-axis-definition#answers>

**Part II - Combine**

Use `groupby()` to sum the financial contribution of wealthy countries in different wealth-score categories across the years surveyed by typing the following code and running it.

```python
data.groupby(wealth_score).sum()
```

## Tidy Data
- [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf) principles are guidlines for organizing data that makes analysis on a computer more efficient and effective.
- Keep the following principles in mind when organizing your data:
  - Each variable has its own column
  - Each observation has its own row
  - Each value must have its own cell (atomic values)
  - Each type of observational unit forms a table

## Long and Wide Formats
- Pandas has several functions that help us rearrange our data when we need to change its structure.
  - This frequently occurs when we need to plot our data.
- When data is in *wide format*, each variable has its own column.
  - Our data is currently in *wide format*

In [ ]:
data.head()

- When data is in *long format*, multiple columns
  are melted into a single columns and entries are repeated.
- To change data to *long format*, `melt()` the columns

In [ ]:
# Country is not currently a variable,
# but an index
long = data.reset_index().melt(id_vars='country', var_name='GDP_Year').sort_values(by='country')
long

- `pivot()` the data to go back to *wide format*

In [ ]:
long.pivot(index='country', columns='GDP_Year')

## Objectives
- Select individual values from a Pandas dataframe.
- Select entire rows or entire columns from a dataframe.
- Select a subset of both rows and columns from a dataframe in a single operation.
- Filter data based on values in a dataframe.
- Apply a split-apply-combine workflow to a dataframe.
- Convert a dataframe between wide and long data formats.